## Set up

In [1]:
import pandas as pd
import requests
from urllib.parse import quote
from ast import literal_eval

In [2]:
df = pd.read_excel(r'C:\Users\ChoJ2\OneDrive - STV Incorporated\Documents - DAS - Projects\Projects\02 Internal Initiatives_R&D\MTA OpenData Challenge 2024\01_Data\Brooklyn_Tract_SpatialSampling_XY_Address.xlsx')

In [3]:
# rename coordinates as lat and lon to reduce confusion
df.rename(columns={'Longitude_Y':'lon','Latitude_X':'lat'},inplace=True)

### Function

In [4]:
# API Call
def get_walk_score(lat, lon, address):
    address = address  # URL encode the address
    url = f"https://api.walkscore.com/score?format=json&address={address}"
    url += f"&lat={lat}&lon={lon}&bike=1&wsapikey=c1c585f79ea03ba285ed6a12c8a87992"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text  # Return the raw JSON response as text
        else:
            return 'Error {}: {}'.format(response.status_code, response.text)
    except requests.exceptions.RequestException as e:
        return f'Error: {e}'

## Get Bike score & description

In [5]:
# API Call
df['api_call'] = df.apply(lambda row: get_walk_score(row['lat'],row['lon'],quote(row['Match_addr'])),axis=1)

In [6]:
# Convert str to dict
df['api_call'] = df['api_call'].apply(literal_eval)

In [7]:
# Parse bike dict
df['bike'] = df.apply(lambda row: row['api_call'].get('bike',None),axis=1)

In [21]:
df.to_excel('API_call_result.xlsx')


In [10]:
# Dropping locations 
bike_score_avil_df = df[~df['bike'].isna()]

In [11]:
# Parse out bike_score
bike_score_avil_df['bike_score'] = bike_score_avil_df.apply(lambda row:row['bike']['score'],axis=1)

In [12]:
# Get average bikescore per Census Tract FIPS
groupby_df = bike_score_avil_df[['Census Tract FIPS Code (2020)','bike_score']].groupby('Census Tract FIPS Code (2020)').mean()
groupby_df

,bike_score
Census Tract FIPS Code (2020),
36047000100,79.0
36047000200,79.6
36047000301,73.4
36047000501,89.2
36047000502,90.6
...,...
36081055500,76.0
36081055900,59.0
36081056100,57.2


From: https://www.walkscore.com/bike-score-methodology.shtml

<table cellspacing="0" cellpadding="0" class="light-dividers align-left">
            <tbody>
              <tr>
                <th><nobr>Bike Score</nobr></th>
                <th>Description</th>
              </tr>
              <tr>
                <td>90–100</td>
                <td><strong>Biker's Paradise</strong><br>
                Daily errands can be accomplished on a bike.</td>
              </tr>
              <tr>
                <td>70–89</td>
                <td><strong>Very Bikeable</strong><br>
                Biking is convenient for most trips.</td>
              </tr>
              <tr>
                <td>50–69</td>
                <td><strong>Bikeable</strong><br>
                  Some bike infrastructure.</td>
              </tr>
              <tr>
                <td>0–49</td>
                <td><strong>Somewhat Bikeable</strong><br>
                Minimal bike infrastructure.</td>
              </tr>
            </tbody>
          </table>

In [13]:
# Add Bike Description
def get_bike_description(score):
    if score >= 90:
        return "Biker's Paradise"
    elif score >= 70:
        return "Very Bikeable"
    elif score >= 50:
        return "Bikeable"
    else: 
        return "Somewhat Bikeable"

In [14]:
groupby_df['description'] = groupby_df.apply(lambda row: get_bike_description(row['bike_score']),axis=1)

In [15]:
# Add Bike Description Detail
def get_bike_detail(score):
    if score >= 90:
        return "Daily errands can be accomplished on a bike."
    elif score >= 70:
        return "Biking is convenient for most trips."
    elif score >= 50:
        return "Some bike infrastructure."
    else: 
        return "Minimal bike infrastructure."

In [16]:
groupby_df['detail'] = groupby_df.apply(lambda row: get_bike_detail(row['bike_score']),axis=1)

In [17]:
groupby_df

,bike_score,description,detail
Census Tract FIPS Code (2020),,,
36047000100,79.0,Very Bikeable,Biking is convenient for most trips.
36047000200,79.6,Very Bikeable,Biking is convenient for most trips.
36047000301,73.4,Very Bikeable,Biking is convenient for most trips.
36047000501,89.2,Very Bikeable,Biking is convenient for most trips.
36047000502,90.6,Biker's Paradise,Daily errands can be accomplished on a bike.
...,...,...,...
36081055500,76.0,Very Bikeable,Biking is convenient for most trips.
36081055900,59.0,Bikeable,Some bike infrastructure.
36081056100,57.2,Bikeable,Some bike infrastructure.


In [18]:
# Export to Excel file
groupby_df.to_excel('bike_score.xlsx')

## Data Validation

In [25]:
df['Census Tract FIPS Code (2020)'].value_counts()

Census Tract FIPS Code (2020)
36047000100    5
36047038200    5
36047036200    5
36047036300    5
36047036400    5
              ..
36061000202    1
36061006200    1
36081055300    1
36081006202    1
36081107202    1
Name: count, Length: 836, dtype: int64

In [20]:
bike_score_avil_df['Census Tract FIPS Code (2020)'].value_counts()

Census Tract FIPS Code (2020)
36047000100    5
36047038200    5
36047036200    5
36047036300    5
36047036400    5
              ..
36061000202    1
36061006200    1
36081055300    1
36081006202    1
36081107202    1
Name: count, Length: 836, dtype: int64